# TextGrad Tutorials: Primitives

![TextGrad](https://github.com/vinid/data/blob/master/logo_full.png?raw=true)

An autograd engine -- for textual gradients!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zou-group/TextGrad/blob/main/examples/notebooks/Prompt-Optimization.ipynb)
[![GitHub license](https://img.shields.io/badge/License-MIT-blue.svg)](https://lbesson.mit-license.org/)
[![Arxiv](https://img.shields.io/badge/arXiv-2406.07496-B31B1B.svg)](https://arxiv.org/abs/2406.07496)
[![Documentation Status](https://readthedocs.org/projects/textgrad/badge/?version=latest)](https://textgrad.readthedocs.io/en/latest/?badge=latest)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/textgrad)](https://pypi.org/project/textgrad/)
[![PyPI](https://img.shields.io/pypi/v/textgrad)](https://pypi.org/project/textgrad/)

**Objectives for this tutorial:**

* Introduce you to the primitives in TextGrad

**Requirements:**

* You need to have an OpenAI API key to run this tutorial. This should be set as an environment variable as OPENAI_API_KEY.


In [2]:
!pip install textgrad # you might need to restart the notebook after installing textgrad
!pip install anthropic
!pip install litellm --upgrade
from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from dotenv import load_dotenv
load_dotenv()
from google.colab import userdata
import os
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

## Introduction: Variable

Variables in TextGrad are the metaphorical equivalent of tensors in PyTorch. They are the primary data structure that you will interact with when using TextGrad.

Variables keep track of gradients and manage the data.

Variables require two arguments (and there is an optional third one):

1. `data`: The data that the variable will hold
2. `role_description`: A description of the role of the variable in the computation graph
3. `requires_grad`: (optional) A boolean flag that indicates whether the variable requires gradients

In [3]:
x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)

In [5]:
x.gradients

set()

## Introduction: Engine

When we talk about the engine in TextGrad, we are referring to an LLM. The engine is an abstraction we use to interact with the model.

In [9]:
# engine = get_engine("gpt-3.5-turbo")
from anthropic import Anthropic
engine = get_engine("claude-3-opus-20240229")

This object behaves like you would expect an LLM to behave: You can sample generation from the engine using the `generate` method.

In [10]:
engine.generate("Hello how are you?")

"Hello! As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you. How may I help you today?"

In [ ]:
# 纯 Anthropic 测试脚本
import os
import anthropic
from google.colab import userdata

# 获取 Key
key = userdata.get('ANTHROPIC_API_KEY')
if not key:
    print("错误：Key 未找到，请检查 Secrets 设置")
else:
    print(f"Key 已加载: {key[:10]}...")

    try:
        client = anthropic.Anthropic(api_key=key)

        # 直接测试 Claude 3.5 Sonnet
        message = client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=100,
            messages=[{"role": "user", "content": "Hello"}]
        )
        print("测试成功！模型回复：", message.content[0].text)

    except anthropic.NotFoundError:
        print("测试失败：Anthropic 找不到该模型。可能是你的账号等级不支持 Sonnet 3.5。")
        print("尝试改用 model='claude-3-opus-20240229' 或 'claude-3-haiku-20240307' 试试？")
    except Exception as e:
        print("发生了其他错误：", e)

## Introduction: Loss

Again, Loss in TextGrad is the metaphorical equivalent of loss in PyTorch. We use Losses in different form in TextGrad but for now we will focus on a simple TextLoss. TextLoss is going to evaluate the loss wrt a string.

In [11]:
system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

## Introduction: Optimizer

Keeping on the analogy with PyTorch, the optimizer in TextGrad is the object that will update the parameters of the model. In this case, the parameters are the variables that have `requires_grad` set to `True`.

**NOTE** This is a text optimizer! It will do all operations with text!

In [12]:
optimizer = TextualGradientDescent(parameters=[x], engine=engine)


## Putting it all together

We can now put all the pieces together. We have a variable, an engine, a loss, and an optimizer. We can now run a single optimization step.

In [13]:
l = loss(x)
l.backward(engine)
optimizer.step()

INFO:textgrad:LLMCall function forward
INFO:textgrad:_backward_through_llm prompt
INFO:textgrad:_backward_through_llm gradient
INFO:textgrad:_backward_through_llm prompt
INFO:textgrad:_backward_through_llm gradient
INFO:textgrad:TextualGradientDescent prompt for update
INFO:textgrad:TextualGradientDescent optimizer response
INFO:textgrad:TextualGradientDescent updated text


In [14]:
x.value

'This sntnece contians several typpos and grammatical errors that a high-quality language model should be capable of detecting and correcting.'

While here it is not going to be useful, we can also do multiple optimization steps in a loop! Do not forget to reset the gradients after each step!

In [17]:
optimizer.zero_grad()